# Experiment on Different Time-series Similarity Measures 1

### Loading

In [1]:
# load image and libraries
%matplotlib inline
import pandas as pd
from matplotlib import cm
import matplotlib.pyplot as plt
import numpy as np
from itertools import groupby
from scipy import signal
from sklearn import preprocessing
from tqdm.notebook import trange, tqdm

root_path = './processed_datasets/'
datasets = ['has', 'sp', 'fp', 'rb', 'sd', 'sr', 'hasb', 'ihas']

# load ground trutht
ori_data_X = []
ori_data_y = []
sketch_X = []
for dataset in datasets:
    file_name = root_path + 'original_' + dataset  
    ori_data_X.append(np.load(file_name + '_X' + '.npy'))
    ori_data_y.append(np.load(file_name + '_y' + '.npy'))
    file_name = root_path + 'sketch_' + dataset + '.npy'
    sketch_X.append(np.load(file_name, allow_pickle=True)[:100])
print(f"number of loaded samples per class: {[len(x) for x in sketch_X]}")
print(f"Original data: {len(ori_data_X)} datasets")
print(f"Sketch data: {len(sketch_X)} datasets")

number of loaded samples per class: [100, 100, 100, 100, 100, 100, 100, 100]
Original data: 8 datasets
Sketch data: 8 datasets


In [2]:
#Smoothing
import pandas as pd
def smoother(series,smoothing):
    series_df = pd.DataFrame(series,columns=['Data'])
    return series_df.ewm(alpha = np.exp(- 3 * smoothing)).mean().to_numpy() 
# test = np.array([1, 2, 3, 4,2,5,2,232,323,23,2,3,23,2,3])
# op = smoother(test,0.5)
# print(op)

In [3]:
#Qetch Algorithm -- inclomplete --
def width(series):
    # Should return width of series -  Size of a 1D array is the same as the length.
    return series.size
    
def height(series):
    #Finds Height of time series based difference in max and minimum values.
    h = np.max(series) - np.min(series) 
    return h


def heightGlobal(series):
    hmax = 0
    hmin = 999
    for i in series:
        hmax = max(np.max(i),hmax)
        hmin = min(np.min(i),hmin)

    h = hmax - hmin
    return h

def widthGlobal(series):
    return series.size

In [4]:
def Split_Correcter(split_arr,h_threshold):
    # Checks if the height is less than 1% of total height and mergers small segments.
    corr_split = []
    p = 0
    buff = []
    split_at = []
    counter = 0
    for i in split_arr:
        if(len(i)==1 or (height(i)<h_threshold)):
            buff.append(i)
        else:
            if(len(buff)>0):
                buff.append(i)
                temp = np.concatenate(buff)
                corr_split.append(temp)

                split_at.append(counter) #Starting position of segment is noted
                counter+=temp.size
                buff = []
            else:
                split_at.append(counter) #Starting position of segment is noted
                counter+=i.size
                corr_split.append(i)

    split_at.append(counter) # Adding the end position of the last segment

    # print("Final",corr_split[:5],"Number of segments:",len(corr_split))
    return corr_split,len(corr_split),split_at

In [5]:
def split_based_derivative(series,c = False):

    if(c):
        series = series.reshape(series.size)

    h_threshold = 0.01 * height(series)
    diff_arr = np.diff(series)
    sign_arr = np.sign(diff_arr)

    # print("THE SIZE IS:",series.shape[0])
    # print("The actual Arr:",series[:50])
    # print("Diff array",diff_arr[:10])
    # print("Sign array",sign_arr[:10])
    p = 0
    split_indices = []
    split_at = []
    for i in range(0,len(sign_arr)):
        if(i==0):
            p = sign_arr[i]
        else:
            if((sign_arr[i] == 0) or (sign_arr[i]==1)):
                if(p==-1):
                    split_indices.append(i)
                    p = sign_arr[i]
            elif((sign_arr[i] == -1) and ((p==1) or (p==0))):
                split_indices.append(i)
                p = sign_arr[i]
                
    # print(series[:10],diff_arr[:10])
    split_arr = np.split(series, split_indices, axis=0)
    # print(len(split_arr))
    # print(len(series))
    # print(series[:10])
    # print(diff_arr[:10])
    # print(sign_arr[:10])
    # print(split_indices)
    # print(split_arr[:3])

    #print("Before Split Correcter")
    corrected_split,k,split_at = Split_Correcter(split_arr,h_threshold)
    return corrected_split,k,split_at


In [6]:
def get_LDE(sketch_split,Candidate_split,Gx,Gy):
    Rx = width(Candidate_split)/(Gx * width(sketch_split))    
    Ry = height(Candidate_split)/(Gy * height(sketch_split))  
    return (np.log(Rx)**2)+(np.log(Ry)**2)

# from scipy.spatial.distance import cityblock
# print(cityblock(x1, x2))

def get_ShapeError(sketch_split,candidate_split,Gy):

    Ni = min(candidate_split.size,sketch_split.size)
    Sum_of_Shape = 0 
    #print("Candidate split", candidate_split,"sketch split", sketch_split, "NI",Ni,"size:",sketch_split.size,candidate_split.size)


    #print("NI",Ni,"sketch and candidate size:",sketch_split.size,candidate_split.size)

    resampled_sketch_split = signal.resample(sketch_split,Ni)
    resampled_candidate_split = signal.resample(candidate_split,Ni)

    Ry = height(resampled_candidate_split)/(Gy * height(resampled_sketch_split)) 


    for i in range(0,Ni):
        Sum_of_Shape += abs(((Gy*Ry*resampled_sketch_split[i]) - resampled_candidate_split[i])/height(candidate_split))        

    return Sum_of_Shape/Ni
    
def calculateDistance(Sketch, Candidate,k):
    Sketch = np.array(Sketch)
    Candidate = np.array(Candidate)

    #print("calculateDistance: Length of the Candidate and Sketch Segment",Candidate.size,Sketch.size,k)

    # Calculating Global non uniform Scaling factors
    Gx = widthGlobal(Candidate)/widthGlobal(Sketch)
    Gy = heightGlobal(Candidate)/heightGlobal(Sketch)
    # Calculating Local distortion and shape errors
    LDE = 0
    SE = 0
    for i in range(0,k-1):
        LDE += get_LDE(Sketch[i],Candidate[i],Gx,Gy)
        SE += get_ShapeError(Sketch[i],Candidate[i],Gy)

    # Calculating total error
    Dist = LDE + SE
    return Dist

In [7]:
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

#### Precision


In [8]:
def result_interpreter_precision(results,curve):

    op = []
    sorted_results = sorted(results, key=lambda x: x[0])

    j = 0
    if(sorted_results[0][0] == 999):
        return [0,0,0]
    while(j<5):
        if(sorted_results[j][3]==curve):
            if(j==0):
                return [1,1,1]
            elif(j<3):
                return [0,1,1]
            elif(j<5):
                return [0,0,1]
        j+=1
    return [0,0,0]

In [ ]:
def qetch_plus_Precision(ori_data_X, ori_data_y, sketch_X, smooth_val_stepsize,curve):
    z = 0
    prec_output = []
    while(z<100):
        results = []
        for i, dataset in enumerate(datasets):
            ResultDistanceObject = []
            original = ori_data_X[i]
            testing_sketch = sketch_X[curve][z]
            #Smoothing by factor of smooth_val_stepsize
            smoothed_candidate_list = []
            smooth_value_list = []
            smooth_value = 0
            while(smooth_value < 1):
                smoothed_candidate_list.append(smoother(original,smooth_value))
                smooth_value_list.append(smooth_value)
                smooth_value += smooth_val_stepsize
            
            Sketch_split_at = []
            split_sketch,k,Sketch_split_at = split_based_derivative(testing_sketch)
            for a in range(len(smoothed_candidate_list)):
                Candidate_split_at = []
                DistanceObject = []
                #Segments Loaded Data into T segments
                split_original,T,Candidate_split_at = split_based_derivative(smoothed_candidate_list[a],True)
               # print("T value and k Value are: ",T,k)
                if(T<k):
                    ResultDistanceObject.append([999,[0,0],smooth_value_list[a],i])
                    print("not possible") #Need to address case where this happens -> Smoothen Sketches with too much K
                    continue
                itr = 0

                while(itr<=T-k):
                    candidate_segments = split_original[itr:k+itr]
                    query_segment = split_sketch
                    itr+=1
                    distance = calculateDistance(query_segment,candidate_segments,k)
                    #Add the starting and ending position Identified
                    start_pos = Candidate_split_at[itr]
                    end_pos = Candidate_split_at[itr+1]
                    DistanceObject.append([distance,[start_pos,end_pos], [a],i])
                # print("Distance object ",DistanceObject)
                ResultDistanceObject.append(min(DistanceObject, key = lambda sublist: sublist[0])) # Will Contain a list of T-k minimum distances
            results.append(min(ResultDistanceObject, key = lambda sublist: sublist[0]))
            print("--- Completed a sketch --- ")
        prec_output.append(result_interpreter_precision(results,curve)) #Should contain 8 best minimum distances
        print("--- Completed a set of sketches --- ")
        z+=1
    print("Prec op:",prec_output)
    s1,s3,s5 = 0,0,0
    for q in prec_output:
        s1+=q[0]
        s3+=q[1]
        s5 += q[2]
    l = len(prec_output)
    s1 = s1/l
    s3 = s3/l
    s5 = s5/l
    precision_op = [s1,s3,s5]

    return precision_op

#datasets = ['has', 'sp', 'fp', 'rb', 'sd', 'sr', 'hasb', 'ihas']
precision = []
smooth_val_stepsize = 0.1
for i in range(0,8):
    precision.append(qetch_plus_Precision(ori_data_X, ori_data_y, sketch_X, smooth_val_stepsize,i))
    print("Completed single type")

In [ ]:
print(precision)

[[0.02, 0.09, 0.24], [0.22, 0.74, 0.88], [0.34, 0.95, 0.98], [0.22, 0.7, 0.76], [0.3, 0.87, 0.96], [0.85, 0.98, 1.0], [0.09, 0.22, 0.46], [0.04, 0.35, 0.8]]


#### Accuracy

In [10]:
# d['Average location error (%)'].append(np.mean([np.abs(x[1][0] - ori_data_y[i][0])/ ori_data_X[i].shape[0]*100 for x in results[i]]))
def result_interpreter_error(results,ori_data_y,curve,length_of_original):

    error = np.mean([np.abs(x[1][0] - ori_data_y[curve][0])/length_of_original*100 for x in results])

    return error

In [12]:
def qetch_plus_accuracy(ori_data_X, ori_data_y, sketch_X, smooth_val_stepsize,curve):
    results = []

    original = ori_data_X[curve]
    length_of_original = original.shape[0]

    #Smoothing by factor of smooth_val_stepsize
    smoothed_candidate_list = []
    smooth_value_list = []
    smooth_value = 0.001
    while(smooth_value < 1):
        smoothed_candidate_list.append(smoother(original,smooth_value))
        smooth_value_list.append(smooth_value)
        smooth_value += smooth_val_stepsize
    # print("smoothed candidate sketches ",len(smoothed_candidate_list),smoothed_candidate_list[0][:10])

    smoothed_sketches = [smoother(sk, 0.3) for sk in sketch_X[curve]]
    # smoothed_sketches = sketch_X[curve]
    for z in trange(100):
        ResultDistanceObject = []
        # testing_sketch = sketch_X[curve][z]
        testing_sketch = smoothed_sketches[z]
        Sketch_split_at = []
        split_sketch,k,Sketch_split_at = split_based_derivative(testing_sketch, True)

        for a in range(0,len(smoothed_candidate_list)):
            Candidate_split_at = []
            #Segments Loaded Data into T segments

            split_original,T,Candidate_split_at = split_based_derivative(smoothed_candidate_list[a],True)

            if(T<k):
                ResultDistanceObject.append([999,[0,0],smooth_value_list[a],curve])
                print("not possible") #Need to address case where this happens -> Smoothen Sketches with too much K
                continue
            
            itr = 0
            DistanceObject = []
            while(itr<=T-k):
                distance = calculateDistance(split_sketch, split_original[itr:k+itr], k)
                #Add the starting and ending position Identified
                start_pos = Candidate_split_at[itr]
                end_pos = Candidate_split_at[itr+1]
                DistanceObject.append([distance,[start_pos,end_pos],smooth_value_list[a],curve])
                itr+=1
            ResultDistanceObject.append(min(DistanceObject, key = lambda sublist: sublist[0])) # Will Contain a list of T-k minimum distances
        results.append(min(ResultDistanceObject, key = lambda sublist: sublist[0])) # Should contain 100 minimum distances.
    print("The set is: ",datasets[curve])
    print("The results looks like: ", results[:2])
    print("Truth value is: ",ori_data_y[curve])
    op = result_interpreter_error(results,ori_data_y,curve,length_of_original)
    print("The error is : ",op)
    return op

smooth_val_stepsize = 0.05
op = qetch_plus_accuracy(ori_data_X, ori_data_y, sketch_X, smooth_val_stepsize,3)

  0%|          | 0/100 [00:00<?, ?it/s]

not possible
not possible
not possible
The set is:  rb
The results looks like:  [[41.222473427311144, [18, 80], 0.051000000000000004, 3], [1.819340438607734, [167, 296], 0.001, 3]]
Truth value is:  [118 246]
The error is :  9.398550724637682


In [13]:
#datasets = ['has', 'sp', 'fp', 'rb', 'sd', 'sr', 'hasb', 'ihas']
error = []
for i in range(0,8):
    error.append(qetch_plus_accuracy(ori_data_X, ori_data_y, sketch_X, smooth_val_stepsize,i))
    print("error is:",error[i])

  0%|          | 0/100 [00:00<?, ?it/s]

not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
The set is:  has
The results looks like:  [[110.58951470837243, [71, 88], 0.501, 0], [26.420141116658357, [66, 128], 0.7010000000000002, 0]]
Truth value is:  [107 243]
The error is :  10.659420289855072
error is: 10.659420289855072


  0%|          | 0/100 [00:00<?, ?it/s]

The set is:  sp
The results looks like:  [[35.55649195111231, [240, 247], 0.201, 1], [34.1380806117899, [247, 256], 0.301, 1]]
Truth value is:  [247 359]
The error is :  6.951690821256038
error is: 6.951690821256038


  0%|          | 0/100 [00:00<?, ?it/s]

...

The set is:  sr
The results looks like:  [[26.104482141513504, [60, 74], 0.40099999999999997, 5], [24.46603291874604, [60, 74], 0.40099999999999997, 5]]
Truth value is:  [170 230]
The error is :  23.23188405797102
error is: 23.23188405797102


  0%|          | 0/100 [00:00<?, ?it/s]

not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
The set is:  hasb
The results looks like:  [[114.36111154606371, [201, 206], 0.201, 6], [33.448091795036234, [0, 28], 0.9010000000000004, 6]]
Truth value is:  [ 80 145]
The error is :  23.862318840579714
error is: 23.862318840579714


  0%|          | 0/100 [00:00<?, ?it/s]

not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
not possible
The set is:  ihas
The results looks like:  [[7.8198711064523, [107, 143], 0.9510000000000004, 7], [30.526326968102573, [165, 169], 0.251, 7]]
Truth value is:  [170 254]
The error is :  14.551807228915658
error is: 14.551807228915658


In [14]:
print(np.mean(error))

14.205751265933301
